In [1]:
import pandas as pd
import glob
from matplotlib import pyplot as plt
%matplotlib inline
import os.path
from collections import defaultdict

In [8]:
# load summary.dat, checking for empty lines
# with open('enet_summary.dat') as fh:
#     lines=fh.read().splitlines()
#     tab=[l.split() for l in lines if len(l.split())==4 ]

# summary=pd.DataFrame(tab, columns=['file','coeffs','cv.rsq','cv.pval'])
# summary['coeffs']=summary['coeffs'].astype('int64')
# summary['cv.rsq']=summary['cv.rsq'].astype('float64')
# summary['cv.pval']=summary['cv.pval'].astype('float64')
summary=pd.read_table('enet_summary.dat',header=None, names=['file','coeffs','cv.rsq','cv.pval'])

In [9]:
len(summary)

8514

In [10]:
summary.head()

,file,coeffs,cv.rsq,cv.pval
0,EUR.ENSG00000000457.wgt.RDat,46,0.001727,0.163376
1,EUR.ENSG00000000938.wgt.RDat,36,-0.001405,0.630785
2,EUR.ENSG00000001084.wgt.RDat,36,-0.001826,0.971351
3,EUR.ENSG00000001460.wgt.RDat,7,-0.001129,0.536719
4,EUR.ENSG00000001561.wgt.RDat,9,-0.001797,0.896094


In [11]:
# sort on cv.pval
sorted_summary=summary.sort_values('cv.pval')
sorted_summary.head()

,file,coeffs,cv.rsq,cv.pval
6812,EUR.ENSG00000198502.wgt.RDat,114,0.735610,2.322327e-160
6920,EUR.ENSG00000204310.wgt.RDat,60,0.713053,1.252123e-150
7832,EUR.ENSG00000236104.wgt.RDat,45,0.682961,8.961344e-139
6935,EUR.ENSG00000204536.wgt.RDat,66,0.667600,3.778735e-133
7392,EUR.ENSG00000223501.wgt.RDat,66,0.661669,4.784117e-131


In [17]:
# add column for just ENSG
sorted_summary['ensg'] = sorted_summary['file'].apply(lambda x: x.split('.')[1])

In [18]:
sorted_summary.to_csv('summary_sorted.dat',sep='\t', index=False)

In [21]:
# genes for which compute_weights function ran but all enet coeffs were 0
all_outputs = glob.glob('/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/*RDat')
all_outputs_genes=[x.split('/')[-1].split('.')[1] for x in all_outputs]
genes_without_models = set(all_outputs_genes)-set(sorted_summary['ensg'])
len(genes_without_models)

7784

In [11]:
# get genes for which compute weights function didn't run
expr_mtx=pd.read_table('/data/cellardata/users/jlz014/TCGA/expression/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.train.txt')
all_brca_genes=list(expr_mtx.gene)

In [ ]:
# list of genes for which compute weights function ran 
genes_ran=[x.split('.')[1] for x in summary.file]+[x.split('.')[1] for x in no_models]
# genes_ran=glob.glob('/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR/*RDat')
# genes_ran=[x.split('.')[1] for x in genes_ran]

In [ ]:
genes_not_ran=set(all_brca_genes)-set(genes_ran)

In [ ]:
# match SGE TASK ID to gene
sge_dict=dict(zip(expr_mtx.gene,range(2,18289)))
no_model_sge=[sge_dict[g] for g in set(all_brca_genes)-set([x.split('.')[1] for x in summary.file])]

In [ ]:
not_run_sge=[sge_dict[g] for g in genes_not_ran]

# Parse table of models with qval<0.1 (input from R)
Map gene names to ENSG

In [13]:
summary = pd.read_table('tcga-brca_enet_models_significant_qval.txt')

In [14]:
# load mapping table for ENSG/ENST/gene name (from Biomart)
gene_map = pd.read_table('/cellar/users/jlz014/Data/ensg_enst_gene_mapping.txt',index_col=0)
gene_map.head()

,Transcript stable ID,Gene name
Gene stable ID,,
ENSG00000276626,ENST00000612820,RF00100
ENSG00000201317,ENST00000364447,RNU4-59P
ENSG00000200823,ENST00000363953,SNORD114-2
ENSG00000221598,ENST00000408671,MIR1249
ENSG00000199595,ENST00000362725,RF00019


In [15]:
ensg_lut=dict(zip(gene_map.index, gene_map['Gene name']))

In [16]:
summary['ensg']=summary['file'].apply(lambda x: x.split('.')[1])
summary['gene']=summary['ensg'].map(ensg_lut)
summary.head()

,file,coeffs,cv.rsq,cv.pval,cv.qval,ensg,gene
0,EUR.ENSG00000204310.wgt.RDat,15,0.731559,7.653302e-163,5.983352e-159,ENSG00000204310,AGPAT1
1,EUR.ENSG00000231925.wgt.RDat,48,0.660028,5.434318e-134,2.124275e-130,ENSG00000231925,TAPBP
2,EUR.ENSG00000163682.wgt.RDat,29,0.658113,2.636727e-133,6.871310e-130,ENSG00000163682,RPL9
3,EUR.ENSG00000198502.wgt.RDat,90,0.647999,9.604646e-130,1.877228e-126,ENSG00000198502,HLA-DRB5
4,EUR.ENSG00000241945.wgt.RDat,116,0.623209,1.977086e-121,3.091372e-118,ENSG00000241945,PWP2


In [17]:
summary.to_csv('tcga-brca_enet_models_significant_qval_with_gene_names.txt',sep='\t', index=False)

# Table with *RDat, tmp plink files (for patients in training set)

In [18]:
MODELS_DIR='/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/EUR'
TMP_DIR='/cellar/users/jlz014/fusion_twas-master/tmp/TCGA_BRCA_EUR_imputed_females_filtered'

In [19]:
models_rdat = glob.glob(os.path.join(MODELS_DIR, '*RDat'))
tmp_bed = glob.glob(os.path.join(TMP_DIR,'*tmp.bed'))
tmp_plink_base = [x[:-4] for x in tmp_bed]

In [4]:
# get ensg for models, generate dicts for lookup
models_ensg = [x.split('.')[-3] for x in models_rdat]
plink_ensg = [x.split('.')[-3] for x in tmp_plink_base]
models_dict = dict(zip(models_ensg, models_rdat))
plink_dict = dict(zip(plink_ensg, tmp_plink_base))

In [20]:
files_lut= pd.DataFrame.from_dict(models_dict,orient='index')

In [21]:
test_plink = ['/cellar/users/jlz014/fusion_twas-master/TCGA-BRCA_normed_weights/brca_test_patients_tmp_plink/TCGA-BRCA.ENSG.chr1-22.median-filt.log2RPKM.qnorm.test.txt.'+x for x in files_lut.index]

In [22]:
files_lut['plink']=test_plink
files_lut.to_csv('tcga-brca_enet_models_files_test_patients.txt',sep='\t', header=None, index=True)

In [23]:
files_lut.head()

,0,plink
ENSG00000243025,/cellar/users/jlz014/fusion_twas-master/TCGA-B...,/cellar/users/jlz014/fusion_twas-master/TCGA-B...
ENSG00000085117,/cellar/users/jlz014/fusion_twas-master/TCGA-B...,/cellar/users/jlz014/fusion_twas-master/TCGA-B...
ENSG00000149084,/cellar/users/jlz014/fusion_twas-master/TCGA-B...,/cellar/users/jlz014/fusion_twas-master/TCGA-B...
ENSG00000104886,/cellar/users/jlz014/fusion_twas-master/TCGA-B...,/cellar/users/jlz014/fusion_twas-master/TCGA-B...
ENSG00000237406,/cellar/users/jlz014/fusion_twas-master/TCGA-B...,/cellar/users/jlz014/fusion_twas-master/TCGA-B...
